In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt
import sklearn
import numpy as np

In [2]:
from sklearn.metrics import roc_curve
from scipy.spatial.distance import cityblock, mahalanobis, euclidean

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
dataset = pd.read_csv(r"C:\Users\asus\Downloads\DSL-StrongPasswordData.csv")

In [5]:
dataset.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [6]:
dataset.describe()

,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
count,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,...,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000,20400.000000
mean,4.500000,25.500000,0.093379,0.264148,0.170769,0.085727,0.169085,0.083358,0.081565,0.159372,...,0.106259,0.150670,0.044411,0.089899,0.202630,0.112731,0.095589,0.321847,0.226259,0.088306
std,2.291344,14.431223,0.029626,0.220534,0.226836,0.027424,0.123546,0.125755,0.026887,0.226928,...,0.038828,0.107422,0.105197,0.030738,0.150191,0.159571,0.028348,0.225389,0.230759,0.027452
min,1.000000,1.000000,0.001400,0.018700,-0.235800,0.009300,0.001100,-0.162100,0.003200,0.001400,...,0.004000,0.001100,-0.235500,0.003700,0.001300,-0.175800,0.003700,0.008300,-0.124500,0.002900
25%,2.750000,13.000000,0.074400,0.146900,0.049800,0.066000,0.113600,0.027200,0.062000,0.089300,...,0.082100,0.096100,-0.009000,0.067300,0.127600,0.023500,0.077400,0.210000,0.114100,0.069900
50%,4.500000,25.500000,0.089500,0.205950,0.108700,0.081000,0.140400,0.057800,0.077100,0.120900,...,0.101900,0.125000,0.022700,0.085300,0.172500,0.095500,0.093700,0.263000,0.160300,0.085500
75%,6.250000,38.000000,0.107900,0.306450,0.212400,0.099800,0.183900,0.096400,0.096900,0.173100,...,0.122300,0.174600,0.068900,0.107900,0.228800,0.145700,0.111100,0.350200,0.255100,0.103700
max,8.000000,50.000000,0.376100,12.506100,12.451700,0.241100,4.919700,4.799900,0.331200,25.987300,...,2.035300,3.327800,2.524200,0.357700,4.025200,3.978200,0.340700,5.883600,5.836400,0.265100


In [7]:
subjects = dataset["subject"].unique()
len(subjects)

51

In [14]:
def evaluateEER(user_scores, impostor_scores):
    labels = [0]*len(user_scores) + [1]*len(impostor_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + impostor_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer

In [15]:
class EuclideanNormedDetector:
    def __init__(self, subjects):
        self.user_scores = []
        self.impostor_scores = []
        self.mean_vector = []
        self.subjects = subjects
        
    def training(self):
        self.mean_vector = self.train.mean().values
        
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = np.linalg.norm(self.test_genuine.iloc[i].values - self.mean_vector)**2
            self.user_scores.append(cur_score)
            
        for i in range(self.test_impostor.shape[0]):
            cur_score = np.linalg.norm(self.test_impostor.iloc[i].values - self.mean_vector)**2
            self.impostor_scores.append(cur_score)
    
    def evaluate(self,dataset):
        eers = []
        
        for subject in subjects:
            
            self.user_scores = []
            self.impostor_scores = []
    
            # Consider current subject as genuine and rest as imposters
            genuine_user_data = dataset.loc[dataset.subject == subject, "H.period":"H.Return"]
            impostor_data = dataset.loc[dataset.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            self.train = genuine_user_data[:200]
    
            # True set (200 records)
            self.test_genuine = genuine_user_data[200:]
    
            # False set (250 records, 5 per imposter, 50 imposters in all)
            self.test_impostor = impostor_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            
            self.training()
            
            self.testing()
    
            eers.append(evaluateEER(self.user_scores, self.impostor_scores))
        
        return np.mean(eers), np.std(eers)

In [11]:
class ManhattanDetector:
    
    def __init__(self, subjects):
        self.user_scores = []
        self.impostor_scores = []
        self.mean_vector = []
        self.subjects = subjects
        
    def training(self):
        self.mean_vector = self.train.mean().values
        
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)
            
        for i in range(self.test_impostor.shape[0]):
            cur_score = cityblock(self.test_impostor.iloc[i].values, self.mean_vector)
            self.impostor_scores.append(cur_score)
    
    def evaluate(self,dataset):
        eers = []
        
        for subject in subjects:
            
            self.user_scores = []
            self.impostor_scores = []
    
            # Consider current subject as genuine and rest as imposters
            genuine_user_data = dataset.loc[dataset.subject == subject, "H.period":"H.Return"]
            impostor_data = dataset.loc[dataset.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            self.train = genuine_user_data[:200]
    
            # True set (200 records)
            self.test_genuine = genuine_user_data[200:]
    
            # False set (250 records, 5 per imposter, 50 imposters in all)
            self.test_impostor = impostor_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            
            self.training()
            
            self.testing()
    
            eers.append(evaluateEER(self.user_scores, self.impostor_scores))
        
        return np.mean(eers), np.std(eers)

In [12]:
class MahalanobisDetector:
    
    def __init__(self, subjects):
        self.user_scores = []
        self.impostor_scores = []
        self.mean_vector = []
        self.subjects = subjects
        
    def training(self):
        self.mean_vector = self.train.mean().values
        self.covinv = np.linalg.inv(np.cov(self.train.T))        
        
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            diff = self.test_genuine.iloc[i].values - self.mean_vector
            cur_score = np.dot(np.dot(diff.T, self.covinv), diff)
            self.user_scores.append(cur_score)
            
        for i in range(self.test_impostor.shape[0]):
            diff = self.test_impostor.iloc[i].values - self.mean_vector
            cur_score = np.dot(np.dot(diff.T, self.covinv), diff)
            self.impostor_scores.append(cur_score)
    
    def evaluate(self,dataset):
        eers = []
        
        for subject in subjects:
            
            self.user_scores = []
            self.impostor_scores = []
    
            # Consider current subject as genuine and rest as imposters
            genuine_user_data = dataset.loc[dataset.subject == subject, "H.period":"H.Return"]
            impostor_data = dataset.loc[dataset.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            self.train = genuine_user_data[:200]
    
            # True set (200 records)
            self.test_genuine = genuine_user_data[200:]
    
            # False set (250 records, 5 per imposter, 50 imposters in all)
            self.test_impostor = impostor_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            
            self.training()
            
            self.testing()
    
            eers.append(evaluateEER(self.user_scores, self.impostor_scores))
        
        return np.mean(eers), np.std(eers)

In [16]:
EuclideanNormedDetector(subjects).evaluate(dataset)

(0.16928649036431045, 0.09313837450860046)

In [17]:
ManhattanDetector(subjects).evaluate(dataset)

(0.15026997944690795, 0.09094241735620097)

In [18]:
MahalanobisDetector(subjects).evaluate(dataset)

(0.19454938055312876, 0.13760435846565805)

In [30]:
class NeuralNetStandardDetector:
    
    def __init__(self, subjects):
        self.user_scores = []
        self.impostor_scores = []
        self.subjects = subjects

    def training(self):
        self.nn = model = keras.Sequential([
            layers.Dense(64, activation='relu', input_dim=31),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')])

        optimizer = tf.keras.optimizers.Adam(0.001)

        self.nn.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mean_absolute_error', 'mean_squared_error'])
 
        self.nn.fit(np.array(self.train), np.ones(self.train.shape[0]),epochs=18)
        
    def testing(self):
        preds = self.nn.predict(np.array(self.test_genuine))
        for tmp in preds:
            self.user_scores.append(tmp)
            
        preds = self.nn.predict(np.array(self.test_impostor))
        for tmp in preds:
            self.impostor_scores.append(tmp)
    
    def evaluate(self,dataset):
        eers = []
        
        for subject in subjects:
            
            self.user_scores = []
            self.impostor_scores = []
    
            # Consider current subject as genuine and rest as imposters
            genuine_user_data = dataset.loc[dataset.subject == subject, "H.period":"H.Return"]
            impostor_data = dataset.loc[dataset.subject != subject, :]
    
            # genuine user's first 200 time vectors for training
            self.train = genuine_user_data[:200]
    
            # True set (200 records)
            self.test_genuine = genuine_user_data[200:]
    
            # False set (250 records, 5 per imposter, 50 imposters in all)
            self.test_impostor = impostor_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
            
            self.training()
            
            self.testing()
    
            eers.append(evaluateEER(self.user_scores, self.impostor_scores))
        
        return np.mean(eers), np.std(eers)

In [31]:
NeuralNetStandardDetector(subjects).evaluate(dataset)

Train on 200 samples
Epoch 1/18
200/200 [==============================] - 5s 23ms/sample - loss: 0.2145 - mean_absolute_error: 0.4625 - mean_squared_error: 0.2145
Epoch 2/18
200/200 [==============================] - 0s 249us/sample - loss: 0.1065 - mean_absolute_error: 0.3200 - mean_squared_error: 0.1065
Epoch 3/18
200/200 [==============================] - 0s 59us/sample - loss: 0.0115 - mean_absolute_error: 0.0940 - mean_squared_error: 0.0115
Epoch 4/18
200/200 [==============================] - 0s 65us/sample - loss: 1.6230e-04 - mean_absolute_error: 0.0095 - mean_squared_error: 1.6230e-04
Epoch 5/18
200/200 [==============================] - 0s 75us/sample - loss: 2.7018e-06 - mean_absolute_error: 0.0011 - mean_squared_error: 2.7018e-06
Epoch 6/18
200/200 [==============================] - 0s 85us/sample - loss: 1.4042e-07 - mean_absolute_error: 2.6388e-04 - mean_squared_error: 1.4042e-07
Epoch 7/18
200/200 [==============================] - 0s 90us/sample - loss: 2.9989e-08 - me

(0.17515383870778917, 0.13937358090117835)

In [ ]:
0.18164394295754704, 0.15237997748329865)